In [1]:
from qick import *
from qick.rfboard import *

import numpy as np
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt

import scipy.io

In [2]:
# Load bitstream with custom overlay
soc = RFQickSoc216V1('./qick_rf216_v1.bit', clk_output=None)
soccfg = soc

print(soccfg)


QICK configuration:

	Board: ZCU216

	Software version: 0.2.226
	Firmware timestamp: Fri Dec 15 13:31:32 2023

	Global clocks (MHz): tProcessor 349.997, RF reference 245.760

	16 signal generator channels:
	0:	axis_sg_mux4_v1 - envelope memory 0 samples (0.000 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 16-bit DDS, range=1228.800 MHz
		DAC tile 0, blk 0 is 0_228, on JHC1
	1:	axis_sg_int4_v1 - envelope memory 4096 samples (13.333 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 16-bit DDS, range=1228.800 MHz
		DAC tile 0, blk 1 is 1_228, on JHC2
	2:	axis_sg_int4_v1 - envelope memory 4096 samples (13.333 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 16-bit DDS, range=1228.800 MHz
		DAC tile 0, blk 2 is 2_228, on JHC1
	3:	axis_sg_int4_v1 - envelope memory 4096 samples (13.333 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 16-bit DDS, range=1228.800 MHz
		DAC tile 0, blk 3 is 3_228, on JHC2
	4:	axis_sg_int4_v1 - envelope memory 4096 samples (13.333 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 16-bit DDS,

In [5]:
##################################################
### Long duration signal for spectrum analysis ###
##################################################
class MuxSGTest(AveragerProgram):
    def initialize(self):
        # Set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"], nqz=1,
                         mixer_freq = self.cfg['mixer_freq'],
                         mux_freqs = self.cfg['pulse_freqs'])        
        
        # Pulse registers.
        length = self.us2cycles(self.cfg['pulse_length'], gen_ch=self.cfg['gen_ch'])
        self.set_pulse_registers(ch = self.cfg['gen_ch'], style = 'const', length=length, mask=[0,1,2,3])
        
        self.synci(200)
    
    def body(self):
        # Trigger.
        self.trigger(pins=[0],t=40)

        # Pulses.
        self.pulse(ch=self.cfg['gen_ch'])
       
config={"gen_ch"      : 0,
        "reps"        : 1,
        "mixer_freq"  : 1400,
        "pulse_length": 10000000, # 10 s
        "pulse_freqs" : [60,250,400,500],
        "period"      : 2,
        "phrst"       : 1,
       }

prog = MuxSGTest(soccfg, config)
prog.config_all(soccfg)

########################
### RF Board Setting ###
########################
nqz = 1

# Set Nyquist Zone.
dacid = soc['gens'][config['gen_ch']]['dac']
soc.rf.set_nyquist(dacid, nqz=nqz)

# Set Filter.
# soc['rfboard']['dacs'][config['gen_ch']].set_filter(fc=7, ftype='highpass')
soc.rfb_set_gen_filter(gen_ch=config['gen_ch'], fc=7, ftype='highpass')

# Set attenuator on DAC.
# soc['rfboard']['dacs'][config['gen_ch']].set_attn_db(attn=0, db=0)
# soc['rfboard']['dacs'][config['gen_ch']].set_attn_db(attn=1, db=10)
soc.rfb_set_gen_rf(gen_ch=config['gen_ch'], att1=0, att2=10)


In [6]:
soc.tproc.start()

In [7]:
soc.gens[0]